# Notebook: 1 Agent Executor

In [7]:
# !pip install langgraph

# Nodes have the function that can be called as needed.In our case Node 1 is starting point and Node2 is our finish point

In [8]:
def function1(input1):
    return input1 + "Hi"

In [13]:
def function2(input2):
    return input2 + "there"

In [14]:
from langgraph.graph import Graph

# Define a langchain graph
workflow = Graph()

workflow.add_node("node_1", function1)
workflow.add_node("node_2", function2)

workflow.add_edge("node_1", "node_2")

workflow.set_entry_point("node_1")
workflow.set_finish_point("node_2")


app = workflow.compile()

In [15]:
app.invoke("Hello")

'HelloHithere'

In [16]:
input = "Hello"
for output in app.stream(input):
    # stream() yields dict with output keyed by node name
    for key,value in output.items():
        print(f"Output from : {key}")
        print("-----")
        print(value)
    print("\n----\n")
    

Output from : node_1
-----
HelloHi

----

Output from : node_2
-----
HelloHithere

----

Output from : __end__
-----
HelloHithere

----



In [17]:
import os 
os.environ["OPENAI_API_KEY"]  = 

In [18]:
from langchain_openai import ChatOpenAI

In [19]:
model = ChatOpenAI(temperature=0)

In [21]:
model.invoke("Hi there").content

'Hello! How can I assist you today?'

# Making Graph with LLM

In [26]:
def function_1(input1):
    response = model.invoke(input1).content
    return response

In [27]:
def function_2(input2):
    return "Agent says : " + input2

In [29]:
#Define a langchain graph
workflow = Graph()

workflow.add_node("agent", function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge("agent", "node_2")
workflow.set_entry_point("agent")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [30]:
app.invoke("what is llm")

'Agent says : LLM stands for Master of Laws, which is an advanced postgraduate degree in law. It is typically pursued by individuals who already have a law degree and wish to specialize in a specific area of law or gain expertise in a particular legal field. The LLM degree is recognized internationally and can lead to career opportunities in various legal sectors.'

In [31]:
input = "what is beautiful way to live life"

for output in app.stream(input):
    for key,value in output.items():
        print(f"Output From {key} : ")
        print("------")
        print(value)
        print("\n----\n")

Output From agent : 
------
A beautiful way to live life is to embrace gratitude, kindness, and positivity. It involves being present in the moment, cherishing the little things, and finding joy in everyday experiences. It also means being true to yourself, following your passions, and cultivating meaningful relationships with others. Living a beautiful life is about finding balance, practicing self-care, and being open to new possibilities and experiences. Ultimately, it is about living authentically and with purpose, making the most of each day and spreading love and light wherever you go.

----

Output From node_2 : 
------
Agent says : A beautiful way to live life is to embrace gratitude, kindness, and positivity. It involves being present in the moment, cherishing the little things, and finding joy in everyday experiences. It also means being true to yourself, following your passions, and cultivating meaningful relationships with others. Living a beautiful life is about finding ba

# Building more complex graph

In [32]:
def function_1(input1):
    prompt = """Your task is to provide the city name based on the user query\n
    Nothing more just the city name mentioned. Following is the user query  : """ + input1
    response = model.invoke(prompt)
    return response.content

In [33]:
def fucntion_2(input2):
    return "Agent says : " + input2


In [34]:
# creating functional graph
workflow = Graph()

workflow.add_node("agent" , function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge("agent", "node_2")

workflow.set_entry_point("agent")
workflow.set_finish_point("node_2")

app = workflow.compile()


In [35]:
app.invoke("What is the weather in kathmandu")

'Agent says : Kathmandu'

In [36]:
input = "What is the weather in Kathmandu"

for output in app.stream(input):
    for key,value in output.items():
        print(f"Output form  {key} :")
        print("------")
        print(value)
        print("\n----\n")
        

Output form  agent :
------
Kathmandu

----

Output form  node_2 :
------
Agent says : Kathmandu

----

Output form  __end__ :
------
Agent says : Kathmandu

----



# Setting up logging 

In [42]:
import os
os.environ["OPENAI_API_KEY"]  = 

In [43]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = 
os.environ["LANGCHAIN_PROJECT"] = "LangGraph_01"

## 1. Making Graphstate

In [46]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
   # The input string
   input: str
   # The list of previous messages in the conversation
   chat_history: list[BaseMessage]
   # The outcome of a given call to the agent
   # Needs `None` as a valid type, since this is what this will start as
   agent_outcome: Union[AgentAction, AgentFinish, None]
   # List of actions and corresponding observations
   # Here we annotate this with `operator.add` to indicate that operations to
   # this state should be ADDED to the existing values (not overwrite it)
   intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

## 2.Creating custom tools

In [47]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool

In [48]:
import random

@tool("lower_case", return_direct=True)
def to_lower_case(input:str) -> str:
  """Returns the input as all lower case."""
  return input.lower()

@tool("random_number", return_direct=True)
def random_number_maker(input:str) -> str:
    """Returns a random number between 0-100."""
    return random.randint(0, 100)

tools = [to_lower_case,random_number_maker]

In [49]:
random_number_maker.run('random')

74

In [50]:
to_lower_case.run('SAM')

'sam'